In [8]:
import os
import numpy as np
import torch

In [9]:
from torch.utils.data import DataLoader
#from generate_data import generate_vrp_data
from utils.functions import load_model
from problem.pdp import PDP

In [10]:
%matplotlib inline
from matplotlib import pyplot as plt

from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D

# Code inspired by Google OR Tools plot:
# https://github.com/google/or-tools/blob/fb12c5ded7423d524fc6c95656a9bdc290a81d4d/examples/python/cvrptw_plot.py

def discrete_cmap(N, base_cmap=None):
  """
    Create an N-bin discrete colormap from the specified input map
    """
  # Note that if base_cmap is a string or None, you can simply do
  #    return plt.cm.get_cmap(base_cmap, N)
  # The following works for string, None, or a colormap instance:

  base = plt.cm.get_cmap(base_cmap)
  color_list = base(np.linspace(0, 1, N))
  cmap_name = base.name + str(N)
  return base.from_list(cmap_name, color_list, N)

def plot_vehicle_routes(data, route, ax1, markersize=5, visualize_demands=False, demand_scale=1, round_demand=False):
    """
    Plot the vehicle routes on matplotlib axis ax1.
    """
    
    # route is one sequence, separating different routes with 0 (depot)
    routes = [r[r!=0] for r in np.split(route.cpu().numpy(), np.where(route==0)[0]) if (r != 0).any()]
    depot = data['depot'].cpu().numpy()
    locs = data['loc'].cpu().numpy()
    demands = data['demand'].cpu().numpy() * demand_scale
    capacity = demand_scale # Capacity is always 1
    
    x_dep, y_dep = depot
    ax1.plot(x_dep, y_dep, 'sk', markersize=markersize*4)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    
    legend = ax1.legend(loc='upper center')
    
    cmap = discrete_cmap(len(routes) + 2, 'nipy_spectral')
    dem_rects = []
    used_rects = []
    cap_rects = []
    qvs = []
    total_dist = 0
    
    #Plot the points with color-matching:
    xs, ys = data['loc'][:, 0].tolist(), data['loc'][:, 1].tolist()
    cmap2 = discrete_cmap(len(xs)//2, 'nipy_spectral')
    node_color = [cmap2(i//2) for i in range(len(xs))]
    
    # Delivery nodes:
    ax1.scatter(xs[::2], ys[::2], color=node_color[::2], s=200, marker='o')
    
    # Pickup nodes:
    ax1.scatter(xs[1::2], ys[1::2], color=node_color[1::2], s=200, marker='X')
    
    
    for veh_number, r in enumerate(routes):
        color = cmap(len(routes) - veh_number) # Invert to have in rainbow order
        route_demands = demands[r - 1]
        coords = locs[r - 1, :]
        xs, ys = coords.transpose()
        total_route_demand = sum(route_demands)
        assert total_route_demand <= capacity
        if not visualize_demands:
            ax1.plot(xs, ys, 'o', mfc=color, markersize=0.00001, markeredgewidth=0.0)
        
        dist = 0
        x_prev, y_prev = x_dep, y_dep
        cum_demand = 0
        for (x, y), d in zip(coords, route_demands):
            dist += np.sqrt((x - x_prev) ** 2 + (y - y_prev) ** 2)
            cap_rects.append(Rectangle((x, y), 0.01, 0.1))
            used_rects.append(Rectangle((x, y), 0.01, 0.1 * total_route_demand / capacity))
            dem_rects.append(Rectangle((x, y + 0.1 * cum_demand / capacity), 0.01, 0.1 * d / capacity))
            
            x_prev, y_prev = x, y
            cum_demand += d
            ax1.text(x_prev, y_prev, '{:.1f} / {}'.format(cum_demand, capacity))
            
        dist += np.sqrt((x_dep - x_prev) ** 2 + (y_dep - y_prev) ** 2)
        total_dist += dist
        qv = ax1.quiver(
            xs[:-1],
            ys[:-1],
            xs[1:] - xs[:-1],
            ys[1:] - ys[:-1],
            scale_units='xy',
            angles='xy',
            scale=1,
            color=color,
            label='R{}, # {}, c {} / {}, d {:.2f}'.format(
                veh_number, 
                len(r), 
                int(total_route_demand) if round_demand else total_route_demand, 
                int(capacity) if round_demand else capacity,
                dist
            )
        )
        
        qvs.append(qv)
        
    ax1.set_title('{} routes, total distance {:.2f}'.format(len(routes), total_dist))
    ax1.legend(handles=qvs)
    
    pc_cap = PatchCollection(cap_rects, facecolor='whitesmoke', alpha=1.0, edgecolor='lightgray')
    pc_used = PatchCollection(used_rects, facecolor='lightgray', alpha=1.0, edgecolor='lightgray')
    pc_dem = PatchCollection(dem_rects, facecolor='black', alpha=1.0, edgecolor='black')
    
    if visualize_demands:
        ax1.add_collection(pc_cap)
        ax1.add_collection(pc_used)
        ax1.add_collection(pc_dem)

In [12]:
model, _ = load_model('outputs/pdp_20/Long_test_with_visited2__20210119T001916/')
torch.manual_seed(1234)
dataset = PDP.make_dataset(filename='data/pdp/pdp20_test_seed1234.pkl', size=20, num_samples=10000)

{'problem': 'pdp', 'graph_size': 20, 'batch_size': 512, 'epoch_size': 1280000, 'val_size': 10000, 'val_dataset': None, 'model': 'attention', 'embedding_dim': 128, 'hidden_dim': 128, 'n_encode_layers': 3, 'tanh_clipping': 10.0, 'normalization': 'batch', 'lr_model': 0.0001, 'lr_critic': 0.0001, 'lr_decay': 1.0, 'eval_only': False, 'n_epochs': 100, 'seed': 1234, 'max_grad_norm': 1.0, 'no_cuda': False, 'exp_beta': 0.8, 'baseline': 'rollout', 'bl_alpha': 0.05, 'bl_warmup_epochs': 1, 'eval_batch_size': 1024, 'checkpoint_encoder': False, 'shrink_size': None, 'data_distribution': None, 'log_step': 50, 'log_dir': 'logs', 'run_name': 'Long_test_with_visited2__20210119T001916', 'output_dir': 'outputs', 'epoch_start': 0, 'checkpoint_epochs': 1, 'load_path': None, 'resume': None, 'no_tensorboard': False, 'no_progress_bar': False, 'use_cuda': True, 'save_dir': 'outputs/pdp_20/Long_test_with_visited2__20210119T001916'}
  [*] Loading model from outputs/pdp_20/Long_test_with_visited2__20210119T001916/e

In [13]:
# Need a dataloader to batch instances
dataloader = DataLoader(dataset, batch_size=1000)

# Make var works for dicts
batch = next(iter(dataloader))

# Run the model
model.eval()
model.set_decode_type('greedy')
with torch.no_grad():
    length, log_p, pi = model(batch, return_pi=True)
tours = pi

# Plot the results
#for i, (data, tour) in enumerate(zip(dataset, tours)):
    #fig, ax = plt.subplots(figsize=(10, 10))
    #plot_vehicle_routes(data, tour, ax, visualize_demands=False, demand_scale=50, round_demand=True)
    # fig.savefig(os.path.join('images', 'cvrp_{}.png'.format(i)))

In [14]:
sum(length) / len(length)

tensor(7.3387)

In [15]:
# Length of the tours:
length

tensor([ 7.6728,  6.5159,  8.8398,  9.1499,  6.1144,  7.9850,  6.6850,  6.8101,
         6.0867,  8.2639,  8.9560,  7.5077,  7.9689,  7.5182,  8.6823,  6.4908,
         7.2999,  7.2153,  6.2967,  8.2255,  6.6066,  6.4070,  7.1152,  7.7022,
         7.6728,  7.0381,  7.5854,  7.0925,  6.1410,  7.9852,  7.7358,  7.8573,
         7.0679,  9.2099,  7.5115,  7.3020,  7.1219,  7.7803,  7.0648,  7.2818,
         6.9455,  9.5196,  7.1413,  6.4493,  7.0136,  8.0382,  9.6272, 10.0713,
         6.6705,  7.9760,  7.1884,  5.4728,  6.0113,  7.2415,  6.6058,  7.9114,
         8.0132,  9.5182,  5.7330,  8.5398,  7.1419,  7.1826,  7.9660,  7.5740,
         6.8125,  7.3068,  6.4699,  6.9686,  6.5766,  5.6760,  8.1464,  6.3578,
         7.4663,  7.8670,  6.2294,  6.3791,  6.5685,  7.8086,  7.6473,  7.2474,
         5.7849,  8.8468,  8.5575,  6.2387,  6.8576,  6.1634,  9.3406,  6.7010,
         9.5244,  6.9713,  8.6588,  6.8492,  5.5632,  7.0085,  7.7810,  6.8200,
         7.4508,  7.0425,  8.5984,  6.75

In [7]:
# Tours found by the agent trained on instances of size 20:
tours

tensor([[17,  3,  5, 18,  4,  7, 13,  9,  6, 11, 12, 14, 15, 10, 16, 19,  1,  8,
         20,  2,  0],
        [13,  9, 17,  7,  8, 11, 14, 12, 10, 18, 19, 20, 15,  3,  4, 16,  1,  2,
          5,  6,  0],
        [19, 17, 11, 20,  1,  9, 12, 18, 10,  2, 15,  7, 16,  5,  8,  6,  3, 13,
         14,  4,  0],
        [ 5, 15, 16,  3,  6,  1,  2,  4,  9, 10,  7,  8, 19, 17, 18, 11, 12, 13,
         14, 20,  0],
        [19,  1, 15,  2, 16,  9, 17, 10, 20, 11,  7, 12,  3, 18,  4,  5,  8,  6,
         13, 14,  0],
        [11, 13,  7, 14,  8,  1, 12,  5,  6,  2, 15, 17,  3, 19, 20, 18,  4, 16,
          9, 10,  0],
        [15,  9, 16,  7, 10,  8, 11,  5, 12,  1,  2,  3,  6,  4, 17, 18, 19, 20,
         13, 14,  0],
        [ 3,  1,  4, 13,  5, 14, 15,  2,  6,  9, 16, 19,  7, 11, 20, 12,  8, 10,
         17, 18,  0],
        [ 7, 19, 20,  8, 11, 15, 16,  9, 10, 12,  5, 17, 18,  6,  3, 13,  1,  4,
         14,  2,  0],
        [ 9, 19,  7,  1,  2,  8, 17, 20, 10, 11, 18, 12,  3,  4,  5, 13, 

In [17]:
# Dataset
dataset[:]

[{'loc': tensor([[0.7837, 0.5631],
          [0.7749, 0.8208],
          [0.2793, 0.6817],
          [0.2837, 0.6567],
          [0.2388, 0.7313],
          [0.6012, 0.3043],
          [0.2548, 0.6294],
          [0.9665, 0.7399],
          [0.4517, 0.4757],
          [0.7842, 0.1525],
          [0.6662, 0.3343],
          [0.7893, 0.3216],
          [0.5247, 0.6688],
          [0.8436, 0.4265],
          [0.9561, 0.0770],
          [0.4108, 0.0014],
          [0.5414, 0.6419],
          [0.2976, 0.7077],
          [0.4189, 0.0655],
          [0.8839, 0.8083]]),
  'demand': tensor([ 0.0667, -0.0667,  0.2667, -0.2667,  0.2000, -0.2000,  0.2667, -0.2667,
           0.0667, -0.0667,  0.4667, -0.4667,  0.0667, -0.0667,  0.3333, -0.3333,
           0.4667, -0.4667,  0.2000, -0.2000]),
  'depot': tensor([0.7528, 0.8988]),
  'type': tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
          1., 0.]),
  'p_or_d': tensor([[0.7749, 0.8208],
          [0.7837, 0.563